In [3]:
# Import dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt


In [4]:
# Set processing unit
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'


In [5]:
# Set seeds of random variables
torch.manual_seed(777)
if device == 'cuda:0':
    torch.cuda.manual_seed_all(777)


In [6]:
# Model training settings
batch_size = 32
n_epochs = 20
lr = 0.01 # learning rate

In [7]:
# MNIST dataset preparation
train_dataset = datasets.MNIST(root='./data/MNIST', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data/MNIST', train=False, download=True, transform=transforms.ToTensor())

# Load MNIST feeder
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

100%|█████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:09<00:00, 1.06MB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 148kB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:01<00:00, 1.14MB/s]
100%|█████████████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<?, ?B/s]


In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels = 1,
                out_channels = 16,
                kernel_size = 5,
                stride = 1,
                padding = 2
            ),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        feature = self.conv1(x) # (N, 28, 28, 1) -> (N, 14, 14, 16)
        feature = self.conv2(feature) # (N, 14, 14, 16) -> (N, 7, 7, 32)
        feature = feature.view(feature.size(0), -1) # (N, 7, 7, 32) -> (N, 7 * 7 * 32)
        output = self.fc(feature) # (N, 7 * 7 * 32) -> (N, 10)
        return output, feature

In [9]:
# Prepare model training
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [10]:
# Visualize the network architecture of a model
print(model)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)


In [11]:
# Train the model
def train(model, train_loader, optimizer, log_interval=5):
    model.train()
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x = x.to(device)
        y = y.to(device)
        output, _ = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (idx + 1) % log_interval == 0:
            print('Train epoch: [{}/{} ({:.2f}%)]\tTrain Loss: {:.6f}'.format(idx * len(x), len(train_loader.dataset), 100. * idx / len(train_loader), loss.item()))

In [12]:
# Evaluate the model
def evaluate(model, test_loader):
    model.eval()
    test_loss = .0
    correct = 0

    with torch.no_grad():
        for x, y in tqdm(test_loader):
            x = x.to(device)
            y = y.to(device)
            output, feature = model(x)
            test_loss += criterion(output, y).item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(y.view_as(prediction)).sum().item()

    test_loss /= len(test_loader.dataset) / batch_size
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy

In [13]:
for epoch in range(n_epochs):
    train(model, train_loader, optimizer, log_interval=10000)
    test_loss, accuracy = evaluate(model, test_loader)
    print("\n[Epoch: {}], \tTest Loss: {:.4f},\tAccuracy: {:.2f} %\n".format(epoch, test_loss, accuracy))

1875it [00:18, 99.31it/s] 
100%|███████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 170.68it/s]



[Epoch: 0], 	Test Loss: 0.0791,	Accuracy: 97.38 %



1875it [00:19, 95.96it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 163.51it/s]



[Epoch: 1], 	Test Loss: 0.0812,	Accuracy: 97.45 %



1875it [00:19, 95.17it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 164.35it/s]


...


[Epoch: 17], 	Test Loss: 0.0988,	Accuracy: 97.33 %



1875it [00:20, 92.38it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 160.44it/s]



[Epoch: 18], 	Test Loss: 0.1033,	Accuracy: 97.32 %



1875it [00:19, 94.02it/s] 
100%|███████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 173.37it/s]


[Epoch: 19], 	Test Loss: 0.0771,	Accuracy: 97.92 %

